In [76]:
import pandas as pd
import numpy as np
import os

# Шлях до файлу
file_path = "D:\\Python\\va\\Energy Indicators.xls"
gdp_file_path = "D:\\Python\\va\\world_bank.csv"
scimen_file_path = "D:\\Python\\va\\ScimEn.xlsx"

# Функція для обробки даних
def answer_one():
    # Читання Excel файлу для енергетичних даних
    energy = pd.read_excel(file_path, skiprows=[*range(0, 18), *range(245, 284)], usecols="C:F", header=None)
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy.replace("...", np.nan, inplace=True)
    energy['Energy Supply'] *= 10**6  # Перетворення в гігаджоули
    energy['Country'] = energy['Country'].str.replace(r'\d+', '', regex=True)
    energy['Country'] = energy['Country'].str.replace(r' \(.*\)', '', regex=True)
    energy['Country'] = energy['Country'].replace(
        ["Republic of Korea", "United States of America", "United Kingdom of Great Britain and Northern Ireland", "China, Hong Kong Special Administrative Region"],
        ["South Korea", "United States", "United Kingdom", "Hong Kong"] )

    # Читання GDP даних
    GDP = pd.read_csv(gdp_file_path, skiprows=5, header=None)
    GDP.columns = ["Country", "Country Code", "Indicator Name", "Indicator Code"] + [str(year) for year in range(1960, 2025)]
    GDP["Country"] = GDP["Country"].replace(
        ["Korea, Rep.", "Iran, Islamic Rep.", "Hong Kong SAR, China"],
        ["South Korea", "Iran", "Hong Kong"] )
    GDP_years = GDP.copy()[["Country"] + [str(year) for year in range(2006, 2016)]]

    # Читання ScimEn даних
    ScimEn = pd.read_excel(scimen_file_path)
    ScimEn = ScimEn[ScimEn['Rank'] <= 15]
    merged_df = ScimEn.merge(energy, on='Country').merge(GDP_years, on='Country')
    merged_df.set_index('Country', inplace=True)

    # Вибір необхідних колонок
    final_df = merged_df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 
                          'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', 
                          '2010', '2011', '2012', '2013', '2014', '2015']]
    
    return final_df

# Виклик функції та виведення результату
TOP15 = answer_one()
print(TOP15)

pd.set_option('future.no_silent_downcasting', True)


                    Rank  Documents  Citable documents  Citations  \
Country                                                             
China                  1     402188             400158    5077232   
United States          2     208042             203356    3540288   
India                  3      81853              79757     968523   
Japan                  4      58342              57735     717946   
United Kingdom         5      56288              54713    1095371   
Germany                6      50906              49773     777362   
Russian Federation     7      46186              45868     217996   
Canada                 8      41209              40390     915491   
Italy                  9      38700              36909     639473   
South Korea           10      38362              38046     618762   
Iran                  11      32080              31725     634135   
France                12      30810              30139     547122   
Spain                 13      2984

In [114]:
def answer_two():
    # Отримуємо DataFrame з топ-15 країн
    df = answer_one()
    
    # Обчислюємо середнє значення ВВП за останні 10 років (2006-2015) для кожної країни
    avgGDP = df.iloc[:, -10:].mean(axis=1, skipna=True)
    
    # Перетворюємо на DataFrame та сортуємо за спаданням
    avgGDP = avgGDP.to_frame(name="Average GDP").sort_values(by="Average GDP", ascending=False)
    
    # Повертаємо результат
    return avgGDP
    
# Викликаємо функцію
avgGDP_for_country = answer_two()
# Виводимо результат
print(avgGDP_for_country)

                     Average GDP
Country                         
United States       1.572243e+13
China               6.927707e+12
Japan               5.239642e+12
Germany             3.590729e+12
United Kingdom      2.777505e+12
France              2.692000e+12
Italy               2.152983e+12
Brazil              1.988889e+12
Russian Federation  1.666746e+12
Canada              1.616359e+12
India               1.602352e+12
Spain               1.406644e+12
South Korea         1.221328e+12
Australia           1.207997e+12
Iran                4.567516e+11


In [118]:
def answer_three():
    # Отримуємо DataFrame з середнім ВВП для кожної країни
    avgGDP_for_country = answer_two()

    # Отримуємо країну з 6-м найбільшим середнім ВВП
    country_sixth = avgGDP_for_country.index[5]  # Індексація починається з 0, тому країна з 6-м найбільшим ВВП буде на індексі 5

    # Отримуємо дані по ВВП для цієї країни на 2006-2015 роки
    country_data = TOP15.loc[country_sixth, '2006':'2015']

    # Обчислюємо зміну ВВП за 10 років (з 2006 по 2015)
    gdp_change = country_data['2015'] - country_data['2006']

    return gdp_change

# Викликаємо функцію та виводимо результат
change_in_gdp = answer_three()
print(change_in_gdp)

124621907951.68018


In [124]:
def answer_four():
    # Додаємо новий стовпець, який є співвідношенням Self-Citations до Total Citations
    TOP15['Self-Citations to Total Citations'] = TOP15['Self-citations'] / TOP15['Citations']
    
    # Знаходимо максимальне значення для нового стовпця
    max_ratio = TOP15['Self-Citations to Total Citations'].max()
    
    # Знаходимо країну з максимальним співвідношенням
    country_with_max_ratio = TOP15['Self-Citations to Total Citations'].idxmax()
    
    return (country_with_max_ratio, max_ratio)

# Викликаємо функцію та виводимо результат
country, ratio = answer_four()
print(f"The country with the highest ratio is {country} with a ratio of {ratio}")


The country with the highest ratio is China with a ratio of 0.6916510807463594


In [126]:
def answer_five():
    # Додаємо новий стовпець для оцінки населення, використовуючи Energy Supply та Energy Supply per Capita
    TOP15['Estimated Population'] = TOP15['Energy Supply'] / TOP15['Energy Supply per Capita']
    
    # Сортуємо країни за оціненим населенням в порядку спадання
    sorted_population = TOP15['Estimated Population'].sort_values(ascending=False)
    
    # Повертаємо третю країну в списку
    third_most_populous_country = sorted_population.index[2]
    
    return third_most_populous_country

# Викликаємо функцію та виводимо результат
third_country = answer_five()
print(f"The third most populous country is {third_country}")


The third most populous country is United States


In [128]:
def answer_six():
    # Створюємо новий стовпець, що оцінює кількість цитованих документів на душу населення
    TOP15['Citable Documents per Person'] = TOP15['Citable documents'] / TOP15['Estimated Population']
    
    # Обчислюємо кореляцію між кількістю цитованих документів на душу населення та енергоспоживанням на душу населення
    correlation = TOP15['Citable Documents per Person'].corr(TOP15['Energy Supply per Capita'])
    
    return correlation

# Викликаємо функцію та виводимо результат
correlation_value = answer_six()
print(f"The correlation between the number of citable documents per capita and the energy supply per capita is: {correlation_value}")


The correlation between the number of citable documents per capita and the energy supply per capita is: 0.7019601723071185


In [130]:
def answer_seven():
    # Словник, який визначає континент для кожної країни
    ContinentDict = {'China':'Asia', 
                     'United States':'North America', 
                     'Japan':'Asia', 
                     'United Kingdom':'Europe', 
                     'Russian Federation':'Europe', 
                     'Canada':'North America', 
                     'Germany':'Europe', 
                     'India':'Asia',
                     'France':'Europe', 
                     'South Korea':'Asia', 
                     'Italy':'Europe', 
                     'Spain':'Europe', 
                     'Iran':'Asia',
                     'Australia':'Australia', 
                     'Brazil':'South America'}

    # Додаємо новий стовпець 'Continent' до таблиці, використовуючи ContinentDict
    TOP15['Continent'] = TOP15.index.map(ContinentDict)

    # Групуємо дані за континентом та обчислюємо необхідні статистики
    continent_stats = TOP15.groupby('Continent')['Estimated Population'].agg(
        size='size', 
        sum='sum', 
        mean='mean', 
        std='std'
    )

    return continent_stats

# Викликаємо функцію та виводимо результат
continent_statistics = answer_seven()
print(continent_statistics)

               size               sum              mean           std
Continent                                                            
Asia              5   2898666386.6106   579733277.32212  6.790979e+08
Australia         1   23316017.316017   23316017.316017           NaN
Europe            6  457929667.216372   76321611.202729  3.464767e+07
North America     2   352855249.48025  176427624.740125  1.996696e+08
South America     1  205915254.237288  205915254.237288           NaN
